In [52]:
import pandas as pd
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt

from mapie.regression import MapieRegressor
from mapie.metrics import regression_coverage_score, regression_mean_width_score
from mapie.regression import MapieRegressor
from mapie.subsample import Subsample

from sklearn.preprocessing import  MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

In [53]:
path = 'data/data.csv'
data = pd.read_csv(path)
normalized_pctg_change = data['normalized_percent_change'] # Save variable fot later use in model
prediction = data['prediction'] 
data.drop(columns=['normalized_percent_change'], inplace=True)

In [54]:
xgboost_df = data.copy()
# One-hot encoding
demographic_vars = ['gender_source_value', 'race_source_value', 'ethnicity_source_value']
xgboost_df = pd.get_dummies(xgboost_df, columns=demographic_vars)
# Scaling: Apparently there's no difference if a use a StandardScaler vs MinMaxScaler
scaler = MinMaxScaler()
numeric_vars = ['mean_led_per_visit', 'age', 'length_of_stay', 'days_since_last_visit', 'days_to_diagnosis']
for i in range(len(numeric_vars)):
    xgboost_df[numeric_vars[i]] = scaler.fit_transform(xgboost_df[[numeric_vars[i]]])

# Reordering the columns so that the target variable is the last one
prediction_to_last = xgboost_df.pop('prediction')
xgboost_df['prediction'] = prediction_to_last

# Defining the features and target variable
X = xgboost_df.iloc[:, :-1] # Shape is rows x features (38)
y = xgboost_df.iloc[:, -1]

# Split data into training and testing sets
# I don't need to use cupy or cudf to send the data to the GPU because I'm using DMatrix
random_state = 21
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
d_train = xgb.DMatrix(X_train, y_train, label=y_train)
d_test = xgb.DMatrix(X_test, y_test, label=y_test)

# Parameter previously calculated by doing hyperparameter tuning
# Note: Accuracy drops with L2 regularization and higher depth. A smaller L1 regularization bumps the model
# Note: When I avoid dropping nas in mean_led_per_patient accuracy bumps to 91%
# Note: -> Remove the least importanf features in the model and run until you find the appropiate number of features
# Model seems to perform better when I use train instead of XGBClassifier
best_params = {
    # Classification
    "eval_metric": "auc", # Area under the curve
    # "eval_metric": "mlogloss" For multiclass
    "objective": "binary:logistic", # Logistic regression for binary classification, output probability
    # "objective": "multi:softprob", # Multi-class classification
    # 'num_class': 3, # Number of classes (required when doing multi-class classification)
    # Regression:
    # "eval_metric": 'rmse',
    # 'objective': 'reg:squarederror',
    'sampling_method': 'gradient_based', # The selection probability for each training instance is proportional to the regularized absolute value of gradients 
    'alpha': 0.1, # L1 regularization
    'lambda': 1, # L2 regularization
    'learning_rate': 0.1, 
    'max_depth': 7,
    'num_boost_round': 700, 
    'tree_method': 'hist', 
    'device': "cuda",
}
num_boost_round = best_params['num_boost_round']
# Prior AUC: 0.86066216116602046
# Best new = 0.92090948515188154 difference is nwq 
regression_params_short = {'alpha': 0.1, 'lambda': 1, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 700, "eval_metric": 'rmse', 'objective': 'reg:squarederror', 'sampling_method': 'gradient_based', 'tree_method': 'hist', 'device': "cuda"} # Updated 28/08/2024. Range (0,1)
regression_params_long = {'alpha': 1, 'lambda': 0.1, 'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 700, "eval_metric": 'rmse', 'objective': 'reg:squarederror', 'sampling_method': 'gradient_based', 'tree_method': 'hist', 'device': "cuda"} # Updated 01/09/2024. Range (-1,1) Decreases coverage in Jackknife+ by 0.02
binary_params = {'alpha': 0, 'lambda': 0.1, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 800, "eval_metric": 'auc', 'objective': 'binary:logistic', 'sampling_method': 'gradient_based', 'tree_method': 'hist', 'device': "cuda",} 

In [55]:
def split_into_five_equal_parts(X, y_binary, y_continuous, random_state=42):
    """
    Split data into 5 equal parts, maintaining alignment between binary and continuous outcomes
    
    Parameters:
    -----------
    X : features DataFrame
    y_binary : binary outcome (change/no change in LEDD)
    y_continuous : continuous outcome (normalized percentage change)
    random_state : for reproducibility
    
    Returns:
    --------
    Training, calibration, and test sets for both binary and continuous outcomes
    """
    # First split: 80% for train+calibration, 20% for test
    X_temp, X_test, y_binary_temp, y_binary_test, y_cont_temp, y_cont_test = train_test_split(
        X, y_binary, y_continuous, test_size=0.2, random_state=random_state
    )
    
    # Split the remaining 80% into 4 equal parts
    X_train, X_temp2, y_binary_train, y_binary_temp2, y_cont_train, y_cont_temp2 = train_test_split(
        X_temp, y_binary_temp, y_cont_temp, test_size=0.75, random_state=random_state
    )
    
    # Split the 60% into 3 equal calibration sets
    X_calib1, X_temp3, y_binary_calib1, y_binary_temp3, y_cont_calib1, y_cont_temp3 = train_test_split(
        X_temp2, y_binary_temp2, y_cont_temp2, test_size=0.666, random_state=random_state
    )
    
    X_calib2, X_calib3, y_binary_calib2, y_binary_calib3, y_cont_calib2, y_cont_calib3 = train_test_split(
        X_temp3, y_binary_temp3, y_cont_temp3, test_size=0.5, random_state=random_state
    )
    
    return (
        X_train, X_calib1, X_calib2, X_calib3, X_test,
        y_binary_train, y_binary_calib1, y_binary_calib2, y_binary_calib3, y_binary_test,
        y_cont_train, y_cont_calib1, y_cont_calib2, y_cont_calib3, y_cont_test
    )

In [56]:
def conformal_prediction_zero_inflated(X, y_binary, y_continuous, alpha_tilda=0.9):
    # Split data into five parts
    (X_train, X_calib1, X_calib2, X_calib3, X_test,
     y_binary_train, y_binary_calib1, y_binary_calib2, y_binary_calib3, y_binary_test,
     y_cont_train, y_cont_calib1, y_cont_calib2, y_cont_calib3, y_cont_test) = split_into_five_equal_parts(
        X, y_binary, y_continuous
    )

    # Convert to DMatrix
    d_train = xgb.DMatrix(X_train, label=y_binary_train)
    d_calib1 = xgb.DMatrix(X_calib1, label=y_binary_calib1)
    d_calib2 = xgb.DMatrix(X_calib2, label=y_binary_calib2)
    d_calib3 = xgb.DMatrix(X_calib3, label=y_binary_calib3)
    d_test = xgb.DMatrix(X_test, label=y_binary_test)
    
    # Train classification model on binary outcome
    classifier = xgb.train(
        params=binary_params,
        dtrain=d_train,
        num_boost_round=binary_params['n_estimators'],
        evals=[(d_train, 'train')],
        early_stopping_rounds=10,
        verbose_eval=False
    )
    
    # Train regression model on non-zero continuous outcomes
    mask_nonzero = y_binary_train == 1
    d_train_reg = xgb.DMatrix(X_train[mask_nonzero], label=y_cont_train[mask_nonzero])
    regressor = xgb.train(
        params=regression_params_short,
        dtrain=d_train_reg,
        num_boost_round=regression_params_short['n_estimators'],
        evals=[(d_train_reg, 'train')],
        early_stopping_rounds=10,
        verbose_eval=False
    )
    
    # First calibration set: Determine alpha_r
    probs_calib1 = classifier.predict(d_calib1)
    r_values = np.arange(0.1, 0.95, 0.05)
    all_interval_lengths = []
    
    for r in r_values:
        alpha_r = np.quantile(probs_calib1, r)
        
        # Second calibration set: Calculate beta_hat
        probs_calib2 = classifier.predict(d_calib2)
        pred_zeros = probs_calib2 <= alpha_r
        beta_hat = np.mean(y_binary_calib2[pred_zeros] == 0)
        
        # Calculate final quantile
        final_quantile = (alpha_tilda - beta_hat * r) / (1 - r)
        if final_quantile < 0 or final_quantile > 1:
            continue
            
        # Third calibration set: Calculate interval width
        probs_calib3 = classifier.predict(d_calib3)
        nonzero_mask = probs_calib3 > alpha_r
        if not any(nonzero_mask):
            continue
            
            
        d_calib3_reg = xgb.DMatrix(X_calib3[nonzero_mask])
        y_pred_nonzero = regressor.predict(d_calib3_reg)
        residuals = np.abs(y_cont_calib3[nonzero_mask] - y_pred_nonzero)
        interval_width = np.quantile(residuals, final_quantile)
        all_interval_lengths.append((r, interval_width, alpha_r))
    
    # Choose best r value and corresponding width
    best_r, best_width, best_alpha_r = min(all_interval_lengths, key=lambda x: x[1])
    
    # Make predictions on test set
    test_probs = classifier.predict(d_test)
    d_test_reg = xgb.DMatrix(X_test)
    test_pred = regressor.predict(d_test_reg)
    
    # Create prediction intervals
    lower_bound = np.zeros_like(test_pred)
    upper_bound = np.zeros_like(test_pred)
    nonzero_mask = test_probs > best_alpha_r
    lower_bound[nonzero_mask] = test_pred[nonzero_mask] - best_width
    upper_bound[nonzero_mask] = test_pred[nonzero_mask] + best_width
    
    return lower_bound, upper_bound, y_cont_test

In [57]:
d_all = xgb.DMatrix(X)
model = xgb.train(best_params, d_train, num_boost_round=num_boost_round, evals=((d_test, "test"),),verbose_eval=True, early_stopping_rounds=10)
y_pred_proba_all = model.predict(d_all, iteration_range=(0, model.best_iteration + 1))
y_pred_all = (y_pred_proba_all > 0.5).astype(int)

[0]	test-auc:0.86294
[1]	test-auc:0.86382
[2]	test-auc:0.87933
[3]	test-auc:0.88895
[4]	test-auc:0.89025
[5]	test-auc:0.89470
[6]	test-auc:0.89596
[7]	test-auc:0.89800
[8]	test-auc:0.89995
[9]	test-auc:0.90051
[10]	test-auc:0.90198
[11]	test-auc:0.90364
[12]	test-auc:0.90498
[13]	test-auc:0.90394
[14]	test-auc:0.90669
[15]	test-auc:0.90791
[16]	test-auc:0.91025
[17]	test-auc:0.91106
[18]	test-auc:0.91245
[19]	test-auc:0.91457
[20]	test-auc:0.91526
[21]	test-auc:0.91669
[22]	test-auc:0.91825
[23]	test-auc:0.91827
[24]	test-auc:0.91895
[25]	test-auc:0.91995
[26]	test-auc:0.92116
[27]	test-auc:0.92150
[28]	test-auc:0.92219
[29]	test-auc:0.92224
[30]	test-auc:0.92256
[31]	test-auc:0.92388
[32]	test-auc:0.92452
[33]	test-auc:0.92480
[34]	test-auc:0.92626
[35]	test-auc:0.92725
[36]	test-auc:0.92796
[37]	test-auc:0.92874
[38]	test-auc:0.92923
[39]	test-auc:0.92959
[40]	test-auc:0.92964
[41]	test-auc:0.92987
[42]	test-auc:0.93013
[43]	test-auc:0.93105
[44]	test-auc:0.93176
[45]	test-auc:0.9320

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:50:12] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)


[48]	test-auc:0.93430
[49]	test-auc:0.93468
[50]	test-auc:0.93605
[51]	test-auc:0.93615
[52]	test-auc:0.93664
[53]	test-auc:0.93707
[54]	test-auc:0.93782
[55]	test-auc:0.93816
[56]	test-auc:0.93814
[57]	test-auc:0.93839
[58]	test-auc:0.93877
[59]	test-auc:0.93946
[60]	test-auc:0.93984
[61]	test-auc:0.94022
[62]	test-auc:0.94035
[63]	test-auc:0.94063
[64]	test-auc:0.94162
[65]	test-auc:0.94183
[66]	test-auc:0.94225
[67]	test-auc:0.94252
[68]	test-auc:0.94300
[69]	test-auc:0.94326
[70]	test-auc:0.94375
[71]	test-auc:0.94465
[72]	test-auc:0.94495
[73]	test-auc:0.94504
[74]	test-auc:0.94561
[75]	test-auc:0.94609
[76]	test-auc:0.94658
[77]	test-auc:0.94715
[78]	test-auc:0.94728
[79]	test-auc:0.94766
[80]	test-auc:0.94787
[81]	test-auc:0.94791
[82]	test-auc:0.94912
[83]	test-auc:0.94931
[84]	test-auc:0.94967
[85]	test-auc:0.94969
[86]	test-auc:0.94961
[87]	test-auc:0.94970
[88]	test-auc:0.94978
[89]	test-auc:0.94994
[90]	test-auc:0.95027
[91]	test-auc:0.95048
[92]	test-auc:0.95081
[93]	test-

In [58]:
lower_bound, upper_bound, y_test = conformal_prediction_zero_inflated(
    X=X,
    y_binary=y_pred_all,
    y_continuous=normalized_pctg_change,
    alpha_tilda=0.9  # Target coverage (90%)
)

# Calculate and print coverage
coverage = np.mean((y_test >= lower_bound) & (y_test <= upper_bound))
interval_width = np.mean(upper_bound - lower_bound)
print(f"Coverage: {coverage:.3f}")
print(f"Average interval width: {interval_width:.3f}")

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:50:13] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:50:14] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Coverage: 0.841
Average interval width: 0.156
